In [59]:
#data source SEC Registered Investment Advisor Report for 2019


#The dataset is supplied via RIA Firms reponses to the SEC's survey found at:
#https://www.sec.gov/about/forms/formadv-part1a.pdf  (This is how the key is deciphered and columns are named appropiately) 




#import dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps



#import API Keys
from config import api_key
from config import gkey

# Dependencies
import json

# Google API Key
from config import gkey

#set census wrapper & gmaps settings

gmaps.configure(api_key = gkey)

#Columns to calculate 
#’Total AUM’ (SUM of 5D(a)(3)-5D(n)(3))
#’State Rank’ (Total AUM ranked by state)
#‘National Rank’ (Total AUM ranked nationally)
#‘Financial planning services and a trust company’ (If columns ‘6A(1) and ‘7A(9)’ true

In [60]:
pd.set_option('display.max_columns', 262)

In [61]:
data = pd.read_excel("Data/SEC Registered Investment Adviser Report 2019-2-1.xlsx")

In [62]:
#data

In [63]:
data['5D(a)(3)'].fillna(0, inplace=True) 
data['5D(b)(3)'].fillna(0, inplace=True) 
data['5D(c)(3)'].fillna(0, inplace=True) 
data['5D(d)(3)'].fillna(0, inplace=True) 
data['5D(e)(3)'].fillna(0, inplace=True) 
data['5D(f)(3)'].fillna(0, inplace=True) 
data['5D(g)(3)'].fillna(0, inplace=True) 
data['5D(h)(3)'].fillna(0, inplace=True) 
data['5D(i)(3)'].fillna(0, inplace=True) 
data['5D(j)(3)'].fillna(0, inplace=True)
data['5D(k)(3)'].fillna(0, inplace=True) 
data['5D(l)(3)'].fillna(0, inplace=True)    
data['5D(m)(3)'].fillna(0, inplace=True) 
data['5D(n)(3)'].fillna(0, inplace=True) 

In [64]:
aums = data[['5D(a)(3)','5D(b)(3)',
             '5D(c)(3)','5D(d)(3)',
             '5D(e)(3)','5D(f)(3)',
             '5D(g)(3)','5D(h)(3)',
             '5D(i)(3)','5D(j)(3)',
             '5D(k)(3)','5D(l)(3)',
             '5D(m)(3)','5D(n)(3)']] 

In [65]:
#aums

In [66]:
data['Total AUM'] = aums.select_dtypes(float).sum(1)

In [67]:
#data.loc[data['Main Office State'] == 'CO']

In [68]:
#data.dtypes

In [69]:
skimmed_columns = data[['Primary Business Name', 
                        'Legal Name', 
                        'Main Office Street Address 1', 
                        'Main Office Street Address 2',
                        'Main Office City',
                        'Main Office State',
                        'Main Office Country',
                        'Total number of offices, other than your Principal Office and place of business',
                        'Website Address',
                        '1O', 
                        '2A(1)',
                        '3A',
                        '5A',
                        '5B(1)',
                        '5B(2)',
                        '5B(5)',
                        '5B(6)',
                        '5D(a)(1)',
                        '5D(a)(2)',
                        '5D(a)(3)',
                        '5D(b)(1)',
                        '5D(b)(2)',
                        '5D(b)(3)',
                        '5D(c)(1)',
                        '5D(c)(2)',
                        '5D(c)(3)',
                        '5D(d)(1)',
                        '5D(d)(3)',
                        '5D(e)(1)',
                        '5D(e)(3)',
                        '5D(f)(1)',
                        '5D(f)(3)',
                        '5D(g)(1)',
                        '5D(g)(2)',
                        '5D(g)(3)',
                        '5D(h)(1)',
                        '5D(h)(2)',
                        '5D(h)(3)',
                        '5D(i)(1)',
                        '5D(i)(2)',
                        '5D(i)(3)',
                        '5D(j)(1)',
                        '5D(j)(2)',
                        '5D(j)(3)',
                        '5D(k)(1)',
                        '5D(k)(2)',
                        '5D(k)(3)',
                        '5D(l)(1)',
                        '5D(l)(2)',
                        '5D(l)(3)',
                        '5D(m)(1)',
                        '5D(m)(2)',
                        '5D(m)(3)',
                        '5D(n)(1)',
                        '5D(n)(2)',
                        '5D(n)(3)',
                        'Total AUM',
                        '5E(1)',
                        '5E(2)',
                        '5E(3)',
                        '5E(4)',
                        '5E(5)',
                        '5E(6)',
                        '5E(7)',
                        '5G(1)',
                        '5I(1)',
                        '6A(1)',
                        '7A(9)',
                        'Total AUM']]  



In [70]:
#skimmed_columns 

In [71]:
columns_renamed = skimmed_columns.rename(index=str, columns={'Total number of offices, other than your Principal Office and place of business':'Total Number of Offices Other Than Main Office',
                        'Website Address':'Website',
                        '1O':'$1 Billion Dollars or More in Assets in Last Fiscal Year', 
                        '2A(1)':'$100 Million Dollars or More Under Manangement', 
                        '3A':'Class of RIA Organization',
                        '5A':'Number of Employees Full Time and Part Time',
                        '5B(1)':'Number of  Employees Who Perform Investment Advisory Functions or Research',
                        '5B(2)':'Number of Employees Registered as a Representative of a Broker-Dealer',
                        '5B(5)':'Number of Employees that are Licensed Agents of an Insurance Company or Agency',
                        '5B(6)':'Number of Firms or Persons that Solicit Advisory Clients on this RIA’s Behalf',
                        '5D(a)(1)':'Number of clients (Individuals)',
                        '5D(a)(2)':'Fewer than 5 clients (Individuals)',
                        '5D(a)(3)':'AUM (Individuals)',
                        '5D(b)(1)':'Number of clients (High Net Worth Individuals)',
                        '5D(b)(2)':'Fewer than 5 clients (High Net Worth Individuals)',
                        '5D(b)(3)':'AUM (High Net Worth Individuals)',
                        '5D(c)(1)':'Number of clients (Banking or Thrift Institutions)',
                        '5D(c)(2)':'Fewer than 5 clients (Banking or Thrift Institutions)',
                        '5D(c)(3)':'AUM (Banking or Thrift Institutions)',
                        '5D(d)(1)':'Number of clients (Investment Companies)',
                        '5D(d)(3)':'AUM (Investment Companies)',
                        '5D(e)(1)':'Number of clients (Business Development Companies)',
                        '5D(e)(3)':'AUM (Business Development Companies)',
                        '5D(f)(1)':'Number of clients (Pooled Investment Vehicles)',
                        '5D(f)(3)':'AUM (Pooled Investment Vehicles)',
                        '5D(g)(1)':'Number of clients (Pension and Profit Sharing Plans)',
                        '5D(g)(2)':'Fewer than 5 clients (Pension and Profit Sharing Plans)',
                        '5D(g)(3)':'AUM (Pension and Profit Sharing Plans)',
                        '5D(h)(1)':'Number of clients (Charitable Organizations)',
                        '5D(h)(2)':'Fewer than 5 clients (Charitable Organizations)',
                        '5D(h)(3)':'AUM (Charitable Organizations)',
                        '5D(i)(1)':'Number of clients (State or Municipal Government Entities)',
                        '5D(i)(2)':'Fewer than 5 clients (State or Municipal Government Entities)',
                        '5D(i)(3)':'AUM (State or Municipal Government Entities)',
                        '5D(j)(1)':'Number of clients (Other Investment Advisers)',
                        '5D(j)(2)':'Fewer than 5 clients (Other Investment Advisers)',
                        '5D(j)(3)':'AUM (Other Investment Advisers)',
                        '5D(k)(1)':'Number of clients (Insurance Companies)',
                        '5D(k)(2)':'Fewer than 5 clients (Insurance Companies)',
                        '5D(k)(3)':'AUM (Insurance Companies)',
                        '5D(l)(1)':'Number of clients (Sovereign Wealth Funds and Foreign Official Institutions)',
                        '5D(l)(2)':'Fewer than 5 clients (Sovereign Wealth Funds and Foreign Official Institutions)',
                        '5D(l)(3)':'AUM (Sovereign Wealth Funds and Foreign Official Institutions)',
                        '5D(m)(1)':'Number of clients (Other Type of Corporation or Business)', 
                        '5D(m)(2)':'Fewer than 5 clients (Other Type of Corporation or Business)',
                        '5D(m)(3)':'AUM (Other Type of Corporation or Business)',
                        '5D(n)(1)':'Number of clients (Other)',
                        '5D(n)(2)':'Fewer than 5 clients (Other)',
                        '5D(n)(3)':'AUM (Other)',
                        '5E(1)':'Compensated by Percentage of AUM',
                        '5E(2)':'Compensated by Hourly Charges',
                        '5E(3)':'Compensated by Subscription Fees via Newsletter or Periodical', 
                        '5E(4)':'Compensated by Fixed Fees Other than Subscription',
                        '5E(5)':'Compensated by Commissions',
                        '5E(6)':'Compensated by Performance-Based Fees',
                        '5E(7)':'Other',
                        '5G(1)':'Renders Financial Planning Services',
                        '5I(1)':'Participates in Wrap Fee Program',
                        '6A(1)':'Actively Engaged in Business as a Trust Company',
                        '7A(9)':'Has Advisory Affiliates Engaged in Business as a Trust Company'}) 

In [72]:
#columns_renamed

In [73]:
columns_renamed.loc[columns_renamed['Main Office Street Address 2'].isnull(), 'Main Office Street Address 2'] = ''


columns_renamed['Main Office Street Address'] = columns_renamed["Main Office Street Address 1"].map(str) + ' ' + columns_renamed["Main Office Street Address 2"].map(str)








In [74]:

columns_renamed.drop('Main Office Street Address 1', axis=1, inplace=True)
columns_renamed.drop('Main Office Street Address 2', axis=1, inplace=True)

In [75]:
columns_renamed
cols = columns_renamed.columns.tolist()
column_to_move = "Main Office Street Address"
new_position = 2

cols.insert(new_position, cols.pop(cols.index(column_to_move)))
columns_renamed = columns_renamed[cols] 








In [76]:
#columns_renamed

In [77]:
columns_renamed['Main Office Street Address'] = columns_renamed['Main Office Street Address'].map(str).map(lambda x: x.strip())
columns_renamed['Main Office City'] = columns_renamed['Main Office City'].map(str).map(lambda x: x.strip())
columns_renamed['Main Office State'] = columns_renamed['Main Office State'].map(str).map(lambda x: x.strip())
columns_renamed['Main Office Country'] = columns_renamed['Main Office Country'].map(str).map(lambda x: x.strip())
   



In [78]:
#columns_renamed

In [79]:
columns_renamed['Main Office Street Address Formatted'] = columns_renamed["Main Office Street Address"].map(str) + ', ' + columns_renamed["Main Office City"].map(str) + ', ' + columns_renamed["Main Office State"].map(str) + ', ' + columns_renamed["Main Office Country"].map(str)





In [80]:
#columns_renamed

In [81]:
columns_renamed
cols = columns_renamed.columns.tolist()
column_to_move = "Main Office Street Address Formatted"
new_position = 2

cols.insert(new_position, cols.pop(cols.index(column_to_move)))
columns_renamed = columns_renamed[cols] 

In [82]:
#columns_renamed

In [83]:
columns_renamed['Main Office Street Address Formatted'] = columns_renamed['Main Office Street Address Formatted'].str.upper()

In [84]:
#columns_renamed

In [85]:
columns_renamed = columns_renamed.loc[:,~columns_renamed.columns.duplicated()]

In [86]:
#columns_renamed

In [87]:
#columns_renamed['Total AUM'] = columns_renamed['Total AUM'].map('{:,.2f}'.format) 



In [88]:
#columns_renamed

In [89]:
#columns_renamed['Total AUM'] = columns_renamed['Total AUM'].map(float).map(lambda x: x.strip())
columns_renamed.dtypes
columns_renamed["Total AUM"] = columns_renamed['Total AUM'].astype(float)

In [90]:
columns_renamed = columns_renamed.sort_values('Total AUM', ascending=False) 

In [91]:
#columns_renamed

In [92]:
columns_renamed = columns_renamed.reset_index()

In [93]:
columns_renamed
del columns_renamed['index']

In [94]:
#columns_renamed

In [95]:
columns_renamed["National Rank"] = columns_renamed["Total AUM"].rank(ascending=False) 

In [96]:
#columns_renamed

In [97]:
del columns_renamed['Fewer than 5 clients (Individuals)']
del columns_renamed['Fewer than 5 clients (High Net Worth Individuals)']
del columns_renamed['Fewer than 5 clients (Banking or Thrift Institutions)']
del columns_renamed['Fewer than 5 clients (Pension and Profit Sharing Plans)']
del columns_renamed['Fewer than 5 clients (Charitable Organizations)']
del columns_renamed['Fewer than 5 clients (State or Municipal Government Entities)']
del columns_renamed['Fewer than 5 clients (Other Investment Advisers)'] 
del columns_renamed['Fewer than 5 clients (Insurance Companies)']
del columns_renamed['Fewer than 5 clients (Sovereign Wealth Funds and Foreign Official Institutions)']
del columns_renamed['Fewer than 5 clients (Other Type of Corporation or Business)']
del columns_renamed['Fewer than 5 clients (Other)']  

In [98]:
#columns_renamed

In [99]:
columns_renamed.loc[columns_renamed['Number of clients (Individuals)'].isnull(), 'Number of clients (Individuals)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (High Net Worth Individuals)'].isnull(), 'Number of clients (High Net Worth Individuals)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Banking or Thrift Institutions)'].isnull(), 'Number of clients (Banking or Thrift Institutions)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Investment Companies)'].isnull(), 'Number of clients (Investment Companies)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Business Development Companies)'].isnull(), 'Number of clients (Business Development Companies)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Pooled Investment Vehicles)'].isnull(), 'Number of clients (Pooled Investment Vehicles)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Pension and Profit Sharing Plans)'].isnull(), 'Number of clients (Pension and Profit Sharing Plans)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Charitable Organizations)'].isnull(), 'Number of clients (Charitable Organizations)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (State or Municipal Government Entities)'].isnull(), 'Number of clients (State or Municipal Government Entities)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Other Investment Advisers)'].isnull(), 'Number of clients (Other Investment Advisers)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Insurance Companies)'].isnull(), 'Number of clients (Insurance Companies)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Sovereign Wealth Funds and Foreign Official Institutions)'].isnull(), 'Number of clients (Sovereign Wealth Funds and Foreign Official Institutions)'] = 0 
columns_renamed.loc[columns_renamed['Number of clients (Other Type of Corporation or Business)'].isnull(), 'Number of clients (Other Type of Corporation or Business)'] = 0  
columns_renamed.loc[columns_renamed['Number of clients (Other)'].isnull(), 'Number of clients (Other)'] = 0 


In [100]:
#columns_renamed

In [101]:
all_clients = columns_renamed[['Number of clients (Individuals)','Number of clients (High Net Worth Individuals)',
             'Number of clients (Banking or Thrift Institutions)','Number of clients (Investment Companies)',
             'Number of clients (Business Development Companies)','Number of clients (Pooled Investment Vehicles)',
             'Number of clients (Pension and Profit Sharing Plans)','Number of clients (Charitable Organizations)',
             'Number of clients (State or Municipal Government Entities)','Number of clients (Other Investment Advisers)',
             'Number of clients (Insurance Companies)','Number of clients (Sovereign Wealth Funds and Foreign Official Institutions)',
             'Number of clients (Other Type of Corporation or Business)','Number of clients (Other)']] 

In [102]:
columns_renamed['Total Clients'] = all_clients.select_dtypes(float).sum(1)

In [103]:
columns_renamed['Average Account AUM'] = columns_renamed['Total AUM'] / columns_renamed['Total Clients'] 

In [104]:
#columns_renamed

In [105]:
co_sec_data = columns_renamed.loc[columns_renamed['Main Office State'] == 'CO'] 

In [106]:
al_sec_data = columns_renamed.loc[columns_renamed['Main Office State'] == 'AL'] 

In [107]:
al_sec_data = al_sec_data.reset_index() 

In [108]:
co_sec_data = co_sec_data.reset_index() 


In [109]:
del co_sec_data['index']
del al_sec_data['index'] 

In [110]:
co_sec_data["State Rank (Based on Total AUM)"] = co_sec_data["Total AUM"].rank(ascending=False)  

In [111]:
pd.set_option('display.max_rows',262)


In [115]:
#co_sec_data

In [113]:
co_sec_data.to_csv('co_sec_data')

In [114]:
co_sec_data.to_json('co_sec_data.json', orient='records', lines=True) 